In [13]:
import torch
import numpy
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(numpy.__version__)

2.7.0+cu118
True
11.8
2.2.5


In [3]:
import numpy as np
print(np.__file__)
print(np.ndarray)

d:\torch_env\lib\site-packages\numpy\__init__.py
<class 'numpy.ndarray'>


In [14]:
import os

# Set Hugging Face cache directories to D: drive
os.environ['TRANSFORMERS_CACHE'] = 'D:/huggingface/cache/transformers'
os.environ['HF_DATASETS_CACHE'] = 'D:/huggingface/cache/datasets'
os.environ['HF_METRICS_CACHE'] = 'D:/huggingface/cache/metrics'
os.environ['HF_HOME'] = 'D:/huggingface/cache/home'

In [10]:
torch.cuda.empty_cache()   

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # or use `load_in_4bit=True`
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    bnb_8bit_compute_dtype=torch.float16,
    device_map="auto"
)

model = AutoModelForCausalLM.from_pretrained(
    "ibm-granite/granite-3.3-2b-instruct",  # or your model
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)


Loading checkpoint shards: 100%|██████████| 2/2 [01:16<00:00, 38.36s/it]


In [ ]:
print(model.hf_device_map)
next(model.parameters()).device

{'': 0}


device(type='cuda', index=0)

In [ ]:
from tqdm import tqdm
import torch
import re
from datasets import load_dataset

# Dataset
dataset = load_dataset("UCSC-VLAA/MedReason", split="train")  # Or "validation"
batch_size = 16
num_samples = len(dataset)

correct = 0
total = 0

option_letters = ['A', 'B', 'C', 'D']

for i in tqdm(range(0, num_samples, batch_size)):
    batch = dataset.select(range(i, min(i + batch_size, num_samples)))

    prompts = []
    answers = []
    for example in batch:
        question = example["question"]
        answer = example["answer"]
        raw_options = example["options"]

        # Fix broken options that are stored as a single string
        if isinstance(raw_options, str):
            # Split by lines and filter only valid A/B/C/D options
            lines = raw_options.strip().splitlines()
            options = []
            for line in lines:
                match = re.match(r"^[A-D]\.\s*(.+)", line)
                if match:
                    options.append(match.group(1).strip())
        else:
            options = raw_options

        if len(options) != 4:
            continue  # Skip bad entries

        answers.append(answer)

        prompt = f"Question: {question.strip()}\n"
        for idx, opt in enumerate(options):
            prompt += f"{option_letters[idx]}. {opt.strip()}\n"
        prompt += "Choose the correct option (A, B, C, or D):"
        prompts.append(prompt)


    try:
        # Tokenize and move to model's device
        print(type(prompts))
        print(type(prompts[0]))
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=2)

        decoded = tokenizer.batch_decode(outputs.cpu(), skip_special_tokens=True)

        for j, output in enumerate(decoded):
            match = re.search(r"\b([A-D])\b", output)
            predicted = match.group(1) if match else ""

            gold_answer = answers[j].strip().upper()
            if predicted == gold_answer:
                correct += 1
            total += 1

    except torch.cuda.OutOfMemoryError:
        print(f"OOM at batch {i}. Skipping.")
        torch.cuda.empty_cache()
        continue

    torch.cuda.empty_cache()

print(f"Accuracy: {correct}/{total} = {correct / total:.2%}")


  0%|          | 0/2043 [00:00<?, ?it/s]

["Question: Urogenital Diaphragm is made up of the following, except:\nA. Deep transverse Perineus\nB. Perinial membrane\nC. Colle's fascia\nD. Sphincter Urethrae\nChoose the correct option (A, B, C, or D):", 'Question: Child with Type I Diabetes. What is the advised time for fundus examinations from the time of diagnosis?\nA. After 5 years\nB. After 2 years\nC. After 10 years\nD. At the time of diagnosis\nChoose the correct option (A, B, C, or D):', 'Question: Most sensitive test for H pylori is-\nA. Fecal antigen test\nB. Biopsy urease test\nC. Serological test\nD. Urea breath test\nChoose the correct option (A, B, C, or D):', 'Question: Ligation of the common hepatic aery will compromise blood flow in\nA. Right and Left gastric aery\nB. Right gastric and sho gastric aeries\nC. Right gastroepiploic and sho gastric aeries\nD. Right gastric and right gastroepiploic aery\nChoose the correct option (A, B, C, or D):', 'Question: Typhoid investigation of choice in 1st week\nA. Blood cultur

  0%|          | 1/2043 [00:01<56:56,  1.67s/it]

['Question: Computed Tomography (CT scan) is least accurate for diagnosts of:\nA. 1cm size Aneurysm in the Hepatic Aery\nB. 1cm size Lymph node inthe para-aoic region\nC. 1cm size Mass in the tail of pancreas\nD. 1cm size Gall stones\nChoose the correct option (A, B, C, or D):', 'Question: NO is synthesized by -\nA. Uracil\nB. Aspartate\nC. Guanosine\nD. Arginine\nChoose the correct option (A, B, C, or D):', 'Question: The serum concentration of which of the following human Ig is maximum:\nA. IgG\nB. IgA\nC. IgM\nD. IgD\nChoose the correct option (A, B, C, or D):', 'Question: Arlts line is seen in\nA. Vernal keratoconjunctis\nB. Pterygium\nC. Ocular pemphigoid\nD. Trachoma\nChoose the correct option (A, B, C, or D):', "Question: A young male patient presents with complete rectal prolapse. The surgery of choice is\nA. Abdominal rectopexy\nB. Delerom's procedure\nC. Anterior resection\nD. Goodsall's procedure\nChoose the correct option (A, B, C, or D):", 'Question: The drug that inhibits

  0%|          | 2/2043 [00:03<1:00:04,  1.77s/it]

...

  4%|▍         | 81/2043 [01:52<50:03,  1.53s/it]

['Question: Drug distribution in tissue true is all except ?\nA. Chloroquine - eye\nB. Ephedrine - bone\nC. Digoxin - skeletal muscle\nD. Minocycline - adipose tissue\nChoose the correct option (A, B, C, or D):', 'Question: A 25-year-old man presents 1 week after discovering that his left testicle is twice the normal size. Physical examination reveals a nontender, testicular mass that cannot be transilluminated. Serum levels of alpha-fetoprotein and human chorionic gonadotropin are normal. A hemiorchiectomy is performed, and histologic examination of the surgical specimen shows the cells grow inalveolar or tubular patterns, the neoplastic cells have an epithelial appearance,exhibit hyperchromatic nuclei with prominent nucleoli.. Compared to normal adult somatic cells, this germ cell neoplasm would most likely show high levels of expression of which of the following proteins?\nA. Desmin\nB. Dystrophin\nC. Cytochrome c\nD. Telomerase\nChoose the correct option (A, B, C, or D):', 'Questio

  4%|▍         | 82/2043 [01:54<45:27,  1.39s/it]

[]


IndexError: list index out of range

In [ ]:
def format_prompt(example):
    return f"""Question: {example['question']}
{example['options']}
Answer:"""

In [8]:
prompt = format_prompt(dataset[0])
print(prompt)

Question: Urogenital Diaphragm is made up of the following, except:
Answer Choices:
A. Deep transverse Perineus
B. Perinial membrane
C. Colle's fascia
D. Sphincter Urethrae
Answer:


In [9]:
%pip install tqdm


The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)

   ---------------------------------------- 0/2 [colorama]
   -------------------- ------------------- 1/2 [tqdm]
   -------------------- ------------------- 1/2 [tqdm]
   -------------------- ------------------- 1/2 [tqdm]
   -------------------- ------------------- 1/2 [tqdm]
   ---------------------------------------- 2/2 [tqdm]

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install sentence-transformers

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl.metadata (29 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.14

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.1 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.
torchvision 0.22.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.7.0 which is incompatible.
